<a href="https://colab.research.google.com/github/spros1/HeartFailureAnalysis/blob/main/ExcerA_Heart_Failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Start the commands in order to load a file from Kaggle to Colab

---


In [7]:
pip install -q kaggle

In [8]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sprospap","key":"9a4854805a9655a32d9fa00b6ca4aa15"}'}

In [9]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d andrewmvd/heart-failure-clinical-data

  0% 0.00/3.97k [00:00<?, ?B/s]
100% 3.97k/3.97k [00:00<00:00, 6.26MB/s]


In [12]:
ls

db.sqlite  heart-failure-clinical-data.zip  kaggle.json  sample_data/


In [13]:
! unzip heart-failure-clinical-data.zip

Archive:  heart-failure-clinical-data.zip
  inflating: heart_failure_clinical_records_dataset.csv  




2.   Check the data and import them to SqlDatabase

---




In [14]:
# Check dataset 
import pandas as pd
from google.colab import files
with open('heart_failure_clinical_records_dataset.csv', 'r') as data:
    df3 = pd.read_csv(data, encoding = ('ansi'))
    print (df3)

      age  anaemia  creatinine_phosphokinase  ...  smoking  time  DEATH_EVENT
0    75.0        0                       582  ...        0     4            1
1    55.0        0                      7861  ...        0     6            1
2    65.0        0                       146  ...        1     7            1
3    50.0        1                       111  ...        0     7            1
4    65.0        1                       160  ...        0     8            1
..    ...      ...                       ...  ...      ...   ...          ...
294  62.0        0                        61  ...        1   270            0
295  55.0        0                      1820  ...        0   271            0
296  45.0        0                      2060  ...        0   278            0
297  45.0        0                      2413  ...        1   280            0
298  50.0        0                       196  ...        1   285            0

[299 rows x 13 columns]


### Create an instance of Database and load to table the CSV

In [22]:
# create the SQLite Database
%pip install csv-to-sqlite

In [23]:
# import to table 
!csv-to-sqlite -f heart_failure_clinical_records_dataset.csv -t full -o db.sqlite

Written 299 rows into 1 tables in 0.018 seconds


### Accessing Database


In [24]:
#Accessing the SQL Database
import sqlite3

In [25]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [26]:
conn = create_connection("db.sqlite")

In [56]:
def select_all(conn, table):
    """
    Query n first rows of the table
    :param conn: the Connection object
    :param table: The table to query
    :param n: Number of rows to query
    """
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM [{table}] order by serum_creatinine" )

    rows = cur.fetchall()
    return rows

In [57]:
def head(conn, table):
  rows = select_all(conn, table)
  for r in rows:
      print(r)

In [ ]:
head(conn, 'heart_failure_clinical_records_dataset')

We now have the data base in SQLite, and some python functions that operate on the data base by wrapping SQL commands.